In [ ]:
import requests

#Nuestras credenciales
client_id = ""
client_secret = ""

#URL para obtencion de tokens
url = "https://accounts.spotify.com/api/token"
#Headers necesarios para la request
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}
#Datos a enviar en la request, aqui van las credenciales
data = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret
}

#Hacemos la request
response = requests.post(url, headers=headers, data=data)

#Guardamos el token
access_token = response.json().get("access_token")

#Mostramos el resultado
print(access_token)

In [ ]:
#Biblioteca para formatear json, sólo necesaria por claridad
import json

#Necesitamos la id del artista para poder obtener su información. Para ésto podemos irnos a un artista en spotify, clickear los tres puntos y copiar su id.
artist_id = "1vCWHaC5f2uS3yhpwWbIA6"
url_artist = f"https://api.spotify.com/v1/artists/{artist_id}"
headers_artist = {
    "Authorization": f"Bearer {access_token}"
}

response_artist = requests.get(url_artist, headers=headers_artist)

print(json.dumps(response_artist.json(), indent=4))